## Setup

In [34]:
from dotenv import load_dotenv

load_dotenv()
llm_config={"model": "gpt-4o-mini"} 

In [16]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-04-23."

## Build groupchat

In [17]:
import autogen

In [18]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [19]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

In [20]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

In [21]:
print(planner.system_message)

Given a task, please determine what information is needed to complete the task. Please note that the information will all be retrieved using Python code. Please only suggest information that can be retrieved using Python code. After each step is done by others, check the progress and instruct the remaining steps. If a step fails, try to workaround


In [22]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

In [23]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [24]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [39]:
groupchat.select_speaker_auto_verbose

False

In [25]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


## Start the group chat!

In [26]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about Nvidia's stock price performance in the past month, we'll need to retrieve and analyze relevant stock market data. Here are the steps to accomplish this using Python:

### Step 1: Retrieve Nvidia's Stock Data
We will use a financial data API, such as Yahoo Finance (`yfinance`), to get Nvidia's stock price data for the past month.

1. **Install the necessary library**:
   ```python
   !pip install yfinance
   ```

2. **Import necessary libraries and fetch the data**:
   ```python
   import yfinance as yf
   import pandas as pd

   # Define the ticker symbol
   ticker_symbol = "NVDA"

   # Determine the date range
   end_date = "2024-04-23"
   start_date = "2024-03-23"

## Add a speaker selection policy

In [30]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [31]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],    
    },
    speaker_transitions_type="allowed",
)

In [37]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = await user_proxy.a_initiate_chat(  # noqa: F704
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
Writer (to chat_manager):

```md
# Nvidia's Stock Price Performance: A Look at the Past Month

As of April 23, 2024, Nvidia Corporation (NASDAQ: NVDA) continues to captivate investors and analysts alike with its ever-evolving stock price. Over the past month, Nvidia's stock has showcased both impressive surges and slight pullbacks, driven by market trends, company announcements, and industry dynamics.

## A Month in Review: Key Movements

### Early April Surge
At the beginning of the month, Nvidia's stock price experienced a noticeable uptick. This rise can be attributed to several factors:

- **Strong Earnings Report**: Nvidia's quarterly earnings report released at the end of March exceeded market expectations. T

    # Nvidia's Stock Price Performance: A Look at the Past Month

As of April 23, 2024, Nvidia Corporation (NASDAQ: NVDA) continues to captivate investors and analysts alike with its ever-evolving stock price. Over the past month, Nvidia's stock has showcased both impressive surges and slight pullbacks, driven by market trends, company announcements, and industry dynamics.

## A Month in Review: Key Movements

![Nvidia Stock Price](nvda_stock_price_apr_2024.png)

### Early April Surge
At the beginning of the month, Nvidia's stock price experienced a noticeable uptick. This rise can be attributed to several factors:

- **Strong Earnings Report**: Nvidia's quarterly earnings report released at the end of March exceeded market expectations. The company reported significant growth in revenue and profitability, bolstered by its data center and gaming divisions.
  
- **AI and Data Center Growth**: The increasing demand for artificial intelligence (AI) and data center solutions has positioned Nvidia as a key player in the tech industry. Announcements of new AI partnerships and data center expansions contributed to investor optimism.

- **Analyst Upgrades**: Several high-profile analysts upgraded their price targets for Nvidia, citing the company’s robust product pipeline and leadership in AI technology. These upgrades spurred positive sentiment in the market.

### Mid-April Volatility
As we moved into mid-April, Nvidia's stock price exhibited some volatility:

- **Market Corrections**: Broader market corrections and profit-taking activities led to fluctuations in Nvidia’s stock. Despite the company’s strong fundamentals, external market factors often play a role in short-term price movements.

- **Geopolitical Concerns**: Ongoing geopolitical tensions, particularly concerning semiconductor supply chains, created uncertainty. While Nvidia is a key innovator, its reliance on global supply chains means it’s not immune to these macroeconomic factors.

### Late April Resilience
In the latter half of April, Nvidia's stock showed remarkable resilience, recovering from mid-month lows:

- **Product Announcements**: Nvidia’s unveiling of new product lines, including the next generation of GPUs and advancements in AI technology, revived investor interest. The anticipation of future growth opportunities led to renewed buying pressure.

- **Technological Milestones**: News of Nvidia’s breakthroughs in quantum computing and collaboration with leading tech firms strengthened market confidence.

## Conclusion: A Promising Outlook
Despite the inherent short-term volatility, the past month has demonstrated Nvidia’s strong position in the tech industry and its potential for continued growth. Investors appear hopeful, given the company’s innovative strides and pivotal role in high-growth sectors like AI and gaming.

As Nvidia continues to navigate the complexities of the global market, the coming months will undoubtedly be exciting for shareholders and market observers alike. Keeping an eye on upcoming earnings reports, product launches, and broader market conditions will be crucial for those looking to understand Nvidia's stock trajectory.

Stay tuned for more updates on Nvidia and other key players in the tech industry.

In [36]:
groupchat_result.cost

{'usage_including_cached_inference': {'total_cost': 0.028274999999999998,
  'gpt-4o-2024-05-13': {'cost': 0.028274999999999998,
   'prompt_tokens': 2868,
   'completion_tokens': 929,
   'total_tokens': 3797}},
 'usage_excluding_cached_inference': {'total_cost': 0.028274999999999998,
  'gpt-4o-2024-05-13': {'cost': 0.028274999999999998,
   'prompt_tokens': 2868,
   'completion_tokens': 929,
   'total_tokens': 3797}}}

In [40]:
groupchat_result.chat_history

[{'content': "Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.",
  'role': 'assistant'},
 {'content': "```md\n# Nvidia's Stock Price Performance: A Look at the Past Month\n\nAs of April 23, 2024, Nvidia Corporation (NASDAQ: NVDA) continues to captivate investors and analysts alike with its ever-evolving stock price. Over the past month, Nvidia's stock has showcased both impressive surges and slight pullbacks, driven by market trends, company announcements, and industry dynamics.\n\n## A Month in Review: Key Movements\n\n### Early April Surge\nAt the beginning of the month, Nvidia's stock price experienced a noticeable uptick. This rise can be attributed to several factors:\n\n- **Strong Earnings Report**: Nvidia's quarterly earnings report released at the end of March exceeded market expectations. The company reported significant growth in revenue and profitability, bolstered by its data center and gaming divisions.\n  \n- **AI

In [43]:
engineer.chat_messages

defaultdict(list,
            {<autogen.agentchat.groupchat.GroupChatManager at 0x1074c9c50>: [{'content': "Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.",
               'name': 'Admin',
               'role': 'user'}],
             <autogen.agentchat.groupchat.GroupChatManager at 0x11eacdb50>: [{'content': "Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.",
               'name': 'Admin',
               'role': 'user'},
              {'content': "```md\n# Nvidia's Stock Price Performance: A Look at the Past Month\n\nAs of April 23, 2024, Nvidia Corporation (NASDAQ: NVDA) continues to captivate investors and analysts alike with its ever-evolving stock price. Over the past month, Nvidia's stock has showcased both impressive surges and slight pullbacks, driven by market trends, company announcements, and industry dynamics.\n\n## A Month in Review: Key Movements\n\n###